In [3]:
import sqlite3

In [4]:
DB_PATH = "Books.db"
connection = sqlite3.connect(DB_PATH)

In [5]:
cursor = connection.cursor()


cursor.execute('''
    CREATE TABLE  IF NOT EXISTS Book(
        id INTEGER PRIMARY KEY,
        author TEXT,
        title TEXT,
        publish_year TEXT
    );
''')


connection.commit()

In [6]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Reader(
        id INTEGER PRIMARY KEY,
        name TEXT
    );
''')


connection.commit()

In [7]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Record(
        book_id INT NOT NULL,
        reader_id INT NOT NULL,
        taking_date TEXT,
        returning_date TEXT,
        FOREIGN KEY (book_id) REFERENCES Book (id),
        FOREIGN KEY (reader_id) REFERENCES Reader (id)
    );
''')


connection.commit()

In [61]:
cursor.execute('''
    DELETE FROM Reader
''')

cursor.execute('''
    INSERT INTO Reader VALUES
        (1, 'Reader 1'),
        (2, 'Reader 2'),
        (3, 'Reader 3');
''')

connection.commit()

In [229]:
cursor.execute('''
    DELETE FROM Book
''')

cursor.execute('''
    INSERT INTO Book VALUES
        (1, 'author 1', 'title 1', '2002'),
        (2, 'author 2', 'title 2', '2002'),
        (3, 'author 3', 'title 3', '2003'),
        (4, 'author 1', 'title 4', '2002'),
        (5, 'author 1', 'title 1', '2004'),
        (6, 'author 4', 'title 5', '2005'),
        (7, 'author 5', 'title 6', '2022');
''')

connection.commit()

In [263]:
cursor = connection.cursor()
cursor.execute('''
    DELETE FROM Record
''')

cursor.execute('''
    INSERT INTO Record VALUES
        (1, 1, '04.03.2010', '04.04.2010'),
        (1, 2, '04.03.2011', '11.22.2011'),
        (2, 5, '04.03.2012', '05.23.2012'),
        (1, 4, '04.03.2010', '04.14.2010'),
        (3, 1, '05.03.2010', '04.03.2011');
''')

connection.commit()

In [64]:
cursor = connection.cursor()

cursor.execute("SELECT * FROM Book WHERE id = 1")
print(cursor.fetchall(), '\n')

cursor.execute("SELECT * FROM Reader WHERE id = 1")
print(cursor.fetchall(), '\n')

cursor.execute("SELECT * FROM Record WHERE book_id = 2")
print(cursor.fetchall(), '\n')

[(1, 'author 1', 'title 1', '2002')] 

[(1, 'Reader 1')] 

[(2, 5, '04.03.2012', '05.23.2012')] 



Вывод списка книг

In [65]:
def list_of_books():
    cursor = connection.cursor()

    book_index = list(cursor.execute("SELECT id FROM Book"))
    for i in book_index:
        cursor.execute("SELECT title FROM Book WHERE id={0}".format(i[0]))
        print(cursor.fetchall()[0][0])

list_of_books()

title 1
title 2
title 3
title 4
title 1
title 5
title 6


Вывод списка читателей

In [67]:
def list_of_readers():
    cursor = connection.cursor()

    reader_index = list(cursor.execute("SELECT id FROM Reader"))
    for i in reader_index:
        cursor.execute("SELECT name FROM Reader WHERE id={0}".format(i[0]))
        print(cursor.fetchall()[0][0])

list_of_readers()

Reader 1
Reader 2
Reader 3


Добавление книги

In [158]:
def add_book(author_str, title_str, publish_year_str):
    cursor = connection.cursor()

    book_index_1 = list(cursor.execute("SELECT id FROM Book"))
    book_index = []
    for i in book_index_1:
        book_index.append(int(i[0]))

    book_index.sort()
    id = book_index[-1] + 1
    print(id)

    cursor.executemany(
        '''INSERT INTO Book(id, author, title, publish_year)
            VALUES (?, ?, ?, ?);''',
        [(id, author_str, title_str, publish_year_str)]
    )

    connection.commit()

In [159]:
add_book('author 1', 'title 4', '2005')

cursor = connection.cursor()
cursor.execute("SELECT * FROM Book")
print(cursor.fetchall())

8
[(1, 'author 1', 'title 1', '2002'), (2, 'author 2', 'title 2', '2002'), (3, 'author 3', 'title 3', '2003'), (4, 'author 1', 'title 4', '2002'), (5, 'author 1', 'title 1', '2004'), (6, 'author 4', 'title 5', '2005'), (7, 'author 5', 'title 6', '2022'), (8, 'author 1', 'title 4', '2005')]


Добавить читателя

In [161]:
def add_reader(name_str):
    cursor = connection.cursor()

    reader_index_1 = list(cursor.execute("SELECT id FROM Reader"))
    reader_index = []
    for i in reader_index_1:
        reader_index.append(int(i[0]))

    reader_index.sort()
    id = reader_index[-1] + 1

    cursor.executemany(
        '''INSERT INTO Reader(id, name)
            VALUES (?, ?);''',
        [(id, name_str)]
    )

    connection.commit()

In [163]:
add_reader('reader_10')

cursor = connection.cursor()
cursor.execute("SELECT * FROM Reader")
print(cursor.fetchall())

[(1, 'Reader 1'), (2, 'Reader 2'), (3, 'Reader 3'), (4, 'reader_10')]


Выдать книгу читателю

In [237]:
def give_book(book_id_int, reader_id_int, taking_date_str, returning_date_str):
    cursor = connection.cursor()

    if type(reader_id_int) != int or type(book_id_int) != int:
        raise TypeError("Need int!")

    reader_index_1 = list(cursor.execute("SELECT id FROM Reader"))
    reader_index = []
    for i in reader_index_1:
        reader_index.append(int(i[0]))

    if reader_id_int not in reader_index:
        raise AttributeError('There is no such reader')

    book_index_1 = list(cursor.execute("SELECT id FROM Book"))
    book_index = []
    for i in book_index_1:
        book_index.append(int(i[0]))

    if book_id_int not in book_index:
        raise AttributeError('There is no such book')

    cursor.execute(
        '''SELECT * FROM Record WHERE book_id = {0} and reader_id = {1}'''.format(book_id_int, reader_id_int)
    )

    if cursor.fetchall() != []:
        raise NameError("Reader have already got this book")

    cursor.executemany(
        '''INSERT INTO Record(book_id, reader_id, taking_date, returning_date)
            VALUES (?, ?, ?, ?);''',
        [(book_id_int, reader_id_int, taking_date_str, returning_date_str)]
    )

    connection.commit()

In [249]:
give_book(5, 1, '10.04.2022', 'None')

cursor = connection.cursor()
cursor.execute("SELECT * FROM Record")
print(cursor.fetchall())

[(1, 1, '04.03.2010', '04.04.2010'), (1, 2, '04.03.2011', '11.22.2011'), (2, 5, '04.03.2012', '05.23.2012'), (1, 4, '04.03.2010', '04.14.2010'), (3, 1, '05.03.2010', '04.03.2011'), (5, 1, '10.04.2022', 'None')]


In [242]:
give_book(5, 1, '10.12.2022', 'None')

cursor = connection.cursor()
cursor.execute("SELECT * FROM Record")
print(cursor.fetchall())

NameError: Reader have already got this book

In [243]:
give_book(10, 3, '10.04.2022', 'None')

AttributeError: There is no such book

In [244]:
give_book(1, 100, '10.04.2022', 'None')

AttributeError: There is no such reader

In [245]:
give_book("1", 100, '10.04.2022', 'None')

TypeError: Need int!

Принять книгу

In [257]:
def get_book(book_id_int, reader_id_int, taking_date_str, returning_date_str):
    cursor = connection.cursor()

    if type(reader_id_int) != int or type(book_id_int) != int:
        raise TypeError("Need int!")

    reader_index_1 = list(cursor.execute("SELECT id FROM Reader"))
    reader_index = []
    for i in reader_index_1:
        reader_index.append(int(i[0]))

    if reader_id_int not in reader_index:
        raise AttributeError('There is no such reader')

    book_index_1 = list(cursor.execute("SELECT id FROM Book"))
    book_index = []
    for i in book_index_1:
        book_index.append(int(i[0]))

    if book_id_int not in book_index:
        raise AttributeError('There is no such book')

    cursor.execute(
        '''DELETE FROM Record WHERE book_id = {0} and reader_id = {1}'''.format(book_id_int, reader_id_int)
    )

    cursor.executemany(
        '''INSERT INTO Record(book_id, reader_id, taking_date, returning_date)
            VALUES (?, ?, ?, ?);''',
        [(book_id_int, reader_id_int, taking_date_str, returning_date_str)]
    )

    connection.commit()

In [258]:
get_book(5, 1, '10.04.2022', '10.04.2023')

cursor = connection.cursor()
cursor.execute("SELECT * FROM Record")
print(cursor.fetchall())

[(1, 1, '04.03.2010', '04.04.2010'), (1, 2, '04.03.2011', '11.22.2011'), (2, 5, '04.03.2012', '05.23.2012'), (1, 4, '04.03.2010', '04.14.2010'), (3, 1, '05.03.2010', '04.03.2011'), (5, 1, '10.04.2022', '10.04.2023')]


In [259]:
def get_book_2(book_id_int, reader_id_int, returning_date_str):
    cursor = connection.cursor()

    if type(reader_id_int) != int or type(book_id_int) != int:
        raise TypeError("Need int!")

    reader_index_1 = list(cursor.execute("SELECT id FROM Reader"))
    reader_index = []
    for i in reader_index_1:
        reader_index.append(int(i[0]))

    if reader_id_int not in reader_index:
        raise AttributeError('There is no such reader')

    book_index_1 = list(cursor.execute("SELECT id FROM Book"))
    book_index = []
    for i in book_index_1:
        book_index.append(int(i[0]))

    if book_id_int not in book_index:
        raise AttributeError('There is no such book')

    cursor.execute(
        '''UPDATE Record
        WHERE book_id = {0} and reader_id = {1}
        SET returning_date = {2}
        '''.format(book_id_int, reader_id_int, returning_date_str)
    )

    connection.commit()

In [264]:
give_book(4, 1, '10.12.2022', 'None')

cursor = connection.cursor()
cursor.execute("SELECT * FROM Record")
print(cursor.fetchall(), '\n\n\n')


get_book(4, 1, '10.04.2022', '10.04.2023')

cursor = connection.cursor()
cursor.execute("SELECT * FROM Record")
print(cursor.fetchall())

[(1, 1, '04.03.2010', '04.04.2010'), (1, 2, '04.03.2011', '11.22.2011'), (2, 5, '04.03.2012', '05.23.2012'), (1, 4, '04.03.2010', '04.14.2010'), (3, 1, '05.03.2010', '04.03.2011'), (4, 1, '10.12.2022', 'None')] 



[(1, 1, '04.03.2010', '04.04.2010'), (1, 2, '04.03.2011', '11.22.2011'), (2, 5, '04.03.2012', '05.23.2012'), (1, 4, '04.03.2010', '04.14.2010'), (3, 1, '05.03.2010', '04.03.2011'), (4, 1, '10.04.2022', '10.04.2023')]
